### Challenge 07 - Margin Leakage in Clinical Supplies - Multi-Source Merging

Our finance team suspects that several high-usage medical products are driving volume but delivering little to no profit margin — or even losses. We’ve observed inconsistencies between usage patterns and expected cost efficiency across several EU countries.

You’ve been given two datasets:

retail_clinical.csv: A transaction-level export of medical product usage (e.g. units billed during clinical operations across 5 countries)
products_clinical.csv: A reference product file that includes product category and base cost per unit

In [111]:
import pandas as pd

### Data loading

In [112]:
products_df = pd.read_csv('../data/products_clinical.csv')
retail_df = pd.read_csv('../data/retail_clinical.csv')

### Data Merging

In [113]:
# Check if dataframes are empty
print(products_df.isnull().sum())
print(retail_df.isnull().sum())

ProductID      0
ProductName    0
Category       0
BaseCost       0
dtype: int64
InvoiceNo      0
ProductID      0
Quantity       0
UnitPrice      0
InvoiceDate    0
Country        0
dtype: int64


In [114]:
# Check datatypes and column names for merge
print(products_df.info())
print(retail_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ProductID    5 non-null      object 
 1   ProductName  5 non-null      object 
 2   Category     5 non-null      object 
 3   BaseCost     5 non-null      float64
dtypes: float64(1), object(3)
memory usage: 288.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    200 non-null    int64  
 1   ProductID    200 non-null    object 
 2   Quantity     200 non-null    int64  
 3   UnitPrice    200 non-null    float64
 4   InvoiceDate  200 non-null    object 
 5   Country      200 non-null    object 
dtypes: float64(1), int64(2), object(3)
memory usage: 9.5+ KB
None


In [115]:
# Merge products dataframe to retail dataframs
products_retail_df = retail_df.merge(products_df, on='ProductID', how='left')

In [116]:
# Check length of dataframe pre- and post- merge are the same
print(len(products_retail_df))
print(len(retail_df))

200
200


In [117]:
print(products_retail_df.isnull().sum())

InvoiceNo      0
ProductID      0
Quantity       0
UnitPrice      0
InvoiceDate    0
Country        0
ProductName    0
Category       0
BaseCost       0
dtype: int64


### Revenue 

#### High-Usage products

In [118]:
# Top 5 quanitity of products sold
quantity = products_retail_df.groupby('ProductName')['Quantity'].sum()
quantity.sort_values(ascending=False)

ProductName
Surgical Mask           419
Insulin Pen             396
COVID Vaccine Dose      384
Rapid Test Kit          354
Antiviral Medication    342
Name: Quantity, dtype: int64

Surgical masks are sold the most, followed by insulin pen, COVID vaccine dose, and rapid test kid

#### Revenue drivers

In [119]:
# Unit price per product
price = products_retail_df.groupby('ProductName')['UnitPrice'].sum()
price.sort_values(ascending=False)

ProductName
Insulin Pen             2463.32
Antiviral Medication    2381.55
Surgical Mask           2329.56
COVID Vaccine Dose      1978.30
Rapid Test Kit          1693.02
Name: UnitPrice, dtype: float64

In [120]:
#  Revenue per product
revenue = quantity * price
revenue.sort_values(ascending=False)

ProductName
Surgical Mask           976085.64
Insulin Pen             975474.72
Antiviral Medication    814490.10
COVID Vaccine Dose      759667.20
Rapid Test Kit          599329.08
dtype: float64

Surgical mask is the product that produces the highest revenue, followed by Insulin pen, and antiviral medication.

In [121]:
basecost = products_retail_df.groupby('ProductName')['BaseCost'].sum()
basecost.sort_values(ascending=False)

ProductName
Antiviral Medication    1800.0
Insulin Pen             1225.0
COVID Vaccine Dose       555.0
Rapid Test Kit           280.0
Surgical Mask             43.0
Name: BaseCost, dtype: float64

In [122]:
cost = quantity * basecost
cost.sort_values(ascending=False)

ProductName
Antiviral Medication    615600.0
Insulin Pen             485100.0
COVID Vaccine Dose      213120.0
Rapid Test Kit           99120.0
Surgical Mask            18017.0
dtype: float64

Antiviral medication is the most expensive to produce

#### Margin analysis

In [132]:
margin = revenue - cost
margin.sort_values(ascending=True)

ProductName
Antiviral Medication    198890.10
Insulin Pen             490374.72
Rapid Test Kit          500209.08
COVID Vaccine Dose      546547.20
Surgical Mask           958068.64
dtype: float64

In [133]:
MarginPerUnit = margin / quantity
MarginPerUnit.sort_values(ascending=True)

ProductName
Antiviral Medication     581.55
Insulin Pen             1238.32
Rapid Test Kit          1413.02
COVID Vaccine Dose      1423.30
Surgical Mask           2286.56
dtype: float64

Top selling low-margin products: 1) Antiviral medication, folllowed by 2) Insulin Pen, 3) Rapid Test Kit

### Country-level slicing

In [125]:
price = products_retail_df.groupby(['Country','ProductName'])['Quantity'].sum()
price.sort_values(ascending=False)

Country         ProductName         
Germany         Surgical Mask           129
United Kingdom  COVID Vaccine Dose      125
Germany         Rapid Test Kit          121
Netherlands     Insulin Pen             101
                COVID Vaccine Dose       89
                Surgical Mask            89
Germany         Antiviral Medication     88
United Kingdom  Antiviral Medication     87
Netherlands     Rapid Test Kit           86
United Kingdom  Insulin Pen              85
Germany         Insulin Pen              84
France          Surgical Mask            78
Spain           COVID Vaccine Dose       76
France          Insulin Pen              74
Netherlands     Antiviral Medication     73
Spain           Surgical Mask            67
United Kingdom  Rapid Test Kit           64
France          COVID Vaccine Dose       58
Spain           Rapid Test Kit           58
United Kingdom  Surgical Mask            56
Spain           Insulin Pen              52
France          Antiviral Medication   